In [1]:
#@title Install kfac_jax

# need version of kfac_jax from after September 13 due to:
# https://github.com/google-deepmind/kfac-jax/issues/142

!pip uninstall kfac-jax -y
!pip install git+https://github.com/google-deepmind/kfac-jax@d3643a1ad85cd34ce9ec096a64c5a44708743217

  Cloning https://github.com/google-deepmind/kfac-jax (to revision d3643a1ad85cd34ce9ec096a64c5a44708743217) to /tmp/pip-req-build-2cbwtl7b
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/kfac-jax /tmp/pip-req-build-2cbwtl7b
  Running command git rev-parse -q --verify 'sha^d3643a1ad85cd34ce9ec096a64c5a44708743217'
  Running command git fetch -q https://github.com/google-deepmind/kfac-jax d3643a1ad85cd34ce9ec096a64c5a44708743217
  Running command git checkout -q d3643a1ad85cd34ce9ec096a64c5a44708743217
  Resolved https://github.com/google-deepmind/kfac-jax to commit d3643a1ad85cd34ce9ec096a64c5a44708743217
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 2.0 MB/s eta 0:00:00
  Created wheel for kfac-jax: filename=kfac_jax-0.0.5-py3-none-any.whl size=145934 sha256=dd9ba6310eb3f12899bdf5b2e01af06abf2eca7dcfead4a0b6d089c8e8504033
  Stored in directory: /root/.cache/pip/wheels/84/00/92/f7138b01929

In [3]:
#@title Import Jax libraries

import flax.linen as nn
import jax
import jax.numpy as jnp
from jax import random

import kfac_jax

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import os
import shutil
import time

In [20]:
d = {}

x = jnp.array([160, 192, 256, 320, 384, 448, 480, 512, 544, 576, 608, 640, 672, 736, 800, 864, 928])

d[1] = jnp.array([2.1390470027923585, 2.117371106147766, 2.079360914230347, 2.0634059429168703, 2.0418279647827147, 2.0355663776397703, 2.026038479804993, 1.9078317523002624, 2.02663938999176, 1.9916373491287231, 2.0125707626342773, 1.710679531097412, 1.6011452078819275, 0.846069610118866, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[2] = jnp.array([2.1376364469528197, 2.1151882171630856, 2.0833156228065492, 2.0677855014801025, 2.0513099908828734, 2.038252031803131, 2.0317952275276183, 1.9605468928813934, 2.004380291700363, 1.8638319730758668, 1.9744050443172452, 1.3116880565881728, 1.7015524208545685, 0.9070954531431198, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[4] = jnp.array([2.137095981836319, 2.1224698305130003, 2.088930642604828, 2.0702469706535336, 2.052493894100189, 2.026791015267372, 2.0324197649955753, 1.9859482169151306, 1.9970914155244825, 1.9276028007268906, 1.974685424566269, 1.305090998113155, 1.2910092800855637, 0.9230422914028167, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[8] = jnp.array([2.1369586765766146, 2.121186754107475, 2.0904825806617735, 2.069430720806122, 2.0525248765945436, 2.0279675677418707, 2.034416487812996, 1.9994710996747016, 1.9973064497113227, 1.8660007819533349, 1.9134346708655356, 1.4452200539410112, 1.2168424412608148, 0.8034091308712958, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[16] = jnp.array([2.1355819702148438, 2.11949605345726, 2.091466601192951, 2.067236562073231, 2.0533283576369286, 2.03372470960021, 2.0313696436584, 1.9970302850008013, 1.8868532452732325, 1.795054268091917, 1.8284942470490932, 1.4987001568078995, 1.2225934054702523, 0.873004288226366, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[32] = jnp.array([2.1353486873209477, 2.121116054803133, 2.0907816089689732, 2.0677708223462106, 2.052898390591144, 2.0356978997588158, 2.0278278812766075, 1.9988414000719787, 1.9362491795793177, 1.8008906293660403, 1.7507613642141222, 1.5744482494890688, 1.14803548976779, 0.891236587241292, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[64] = jnp.array([2.137277441099286, 2.120296173915267, 2.0904221937060354, 2.068283974006772, 2.053763113170862, 2.0363531768321987, 2.0256495762616398, 2.0059976944699884, 1.9559663173742594, 1.8503267159685493, 1.705769655108452, 1.524729472398758, 1.1931680483743548, 0.924496098794043, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[128] = jnp.array([2.135964014567435, 2.119187116064131, 2.0901578743010756, 2.0690575897693635, 2.053141443990171, 2.0379311349242926, 2.0286932651884855, 2.006364729627967, 1.9611748593859377, 1.8800712652504445, 1.6749626398552209, 1.5216509925667197, 1.266984701436013, 0.9443971992935987, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[256] = jnp.array([2.1367662544362247, 2.11862374022603, 2.0895097734406587, 2.0692675811238583, 2.0531316876411436, 2.037918024184182, 2.0288481385912744, 2.008296172041446, 1.9522120296023786, 1.8683750097174197, 1.6537404564907776, 1.4866096682846546, 1.2687536558369175, 0.955623487662524, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[512] = jnp.array([2.137574161682278, 2.118415069114417, 2.0890740695409473, 2.0693489262834186, 2.0537336371839046, 2.0383666599867865, 2.0280791949713604, 2.0088493178132922, 1.9522541341837496, 1.8633897365187293, 1.6931888234918007, 1.5011318034026773, 1.2837497161002829, 0.9645728270639665, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[1024] = jnp.array([2.1381775154266505, 2.118135644472204, 2.0895444331225006, 2.0699078844860193, 2.054174785851501, 2.038330789306201, 2.028024197195191, 2.005973980214913, 1.9558538354118356, 1.863138405947356, 1.708915365703837, 1.5065040085813963, 1.2983537114982027, 0.9589719491399592, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])
d[2048] = jnp.array([2.138090460931993, 2.1182356783538125, 2.0895550782467835, 2.0698839781462883, 2.0540498701977157, 2.038370887782952, 2.0281428066222635, 2.004968715856055, 1.9557614909132588, 1.863138405947356, 1.708915365703837, 1.5063766357504969, 1.2975472073862941, 0.9655968356014688, 0.758564773053006, 0.6476833794649804, 0.5569544512818092])


In [21]:
#@title The functional forms

def count_params(params):
  return sum([jnp.prod(jnp.arratarget(x.shape))
              for x in jax.tree_util.tree_leaves(params)])

def bnsl_dim(x, n_breaks, name):
  out = nn.Dense(1)(x) + nn.Dense(1, use_bias=False)(nn.softplus(type(name, (nn.Dense,), {})(n_breaks)(x)))
  return out

class FunctionalForm(nn.Module):
  """BNSL operating in log-log space (i.e. inputs (and children of inputs) are never explicitly raised to a power)."""
  n_breaks: int

  @nn.compact
  def __call__(self, x, batch_train):
    x = jnp.log(x)

    offset = 1e-16

    x_train, y_train = batch_train
    x_train, y_train = jnp.log(x_train), jnp.log(y_train + offset)

    x_mean = jnp.mean(x_train, axis=0, keepdims=True)
    x_std = jnp.std(x_train, axis=0, keepdims=True)
    x_std = jnp.where(x_std == 0., 1., x_std)
    x = (x - x_mean)/x_std

    y_mean = jnp.mean(y_train)
    u = jnp.ones_like(x[:,0]) * y_mean

    eps_2 = jnp.log(1e-20)

    eps_array = jnp.ones_like(x) * eps_2

    x0 = jnp.zeros_like(x)

    a = type("irr_ent", (nn.Dense,), {})(1, use_bias=True)(x0)

    _mbnsl = jnp.concatenate([
          bnsl_dim(x, self.n_breaks, 'x'),
          a,
          eps_array,
      ], axis=-1)

    mbnsl = jax.scipy.special.logsumexp(_mbnsl, axis=-1, keepdims=False)

    u = mbnsl + u

    return jnp.exp(mbnsl)

def functional_form__sle(params, _model, batch, batch_train):
    x, y = batch
    y_pred = _model.apply(params, x, batch_train)
    pred, target = jnp.log(y_pred), jnp.log(y)
    return jnp.square(pred - target)

def functional_form__standard_le(params, _model, batch, batch_train):
    x, y = batch
    y_pred = _model.apply(params, x, batch_train)
    pred, target = jnp.log(y_pred), jnp.log(y)

    error = (pred - target) ** 2
    err_mu = jnp.mean(error)
    std_err = jnp.sqrt(err_mu + jnp.std(error) / (len(pred)**0.5)) - jnp.sqrt(err_mu)

    return std_err


In [ ]:
#@title Fits the functional form

try:
  shutil.rmtree("plots")
except:
  pass
os.mkdir("plots")

data_dict = {
    "name": [],
    "rmsle_train": [],
    "rmsle_extrap": [],
    "rmsle_all": [],
    "standard_le_train": [],
    "standard_le_all": [],
}


for key, value in d.items():
    y = value

    title = str("4 Digit Addition (")+str(key)+" "+str("seeds)")
    plot_name = title
    plot_name = plot_name.replace(" ", "_")

    x_train = []
    x_test = []

    y_train = []
    y_test = []

    x_split = 750

    for _x, _y in zip(x, y):
      if _x < x_split:
        x_train.append(_x)
        y_train.append(_y)
      else:
        x_test.append(_x)
        y_test.append(_y)

    x_train = jnp.expand_dims(jnp.array(x_train), -1)
    x_test = jnp.expand_dims(jnp.array(x_test), -1)

    y_train = jnp.array(y_train)
    y_test = jnp.array(y_test)

    x_all = jnp.expand_dims(jnp.array(x), -1)
    y_all = y

    n_breaks = 1

    model = FunctionalForm(n_breaks=n_breaks)
    params = model.init(random.PRNGKey(21), x_train, (x_train, y_train))

    def loss(params, batch, offset=1e-16):

      x, y = batch
      y_pred = model.apply(params, x, batch)

      pred, target = jnp.log(offset + y_pred), jnp.log(offset + y)

      y_mean = jnp.mean(target)
      y_std = jnp.std(target)
      y_std = jnp.where(y_std == 0., 1., y_std)

      target = target / y_std
      pred = pred / y_std

      kfac_jax.register_squared_error_loss(pred, target)

      return jnp.mean(jnp.square(pred - target))
    loss_and_grad = jax.value_and_grad(loss, argnums=0)

    # using a second order optimizer makes training way faster
    optimizer = kfac_jax.Optimizer(
      value_and_grad_func=loss_and_grad,
      l2_reg=0.0, # 1e-4,
      value_func_has_aux=False,
      value_func_has_state=False,
      value_func_has_rng=False,
      use_adaptive_learning_rate=True,
      use_adaptive_momentum=True,
      use_adaptive_damping=True,
      initial_damping=1.0,
      multi_device=False,
    )

    rng = random.PRNGKey(0)
    rng, init_rng = random.split(rng)
    opt_state = optimizer.init(params, init_rng, (x_train, y_train))

    start_time = time.time()

    # Fits in <1min on 1 GPU
    best_params, best_loss = params, 1e6

    for j in range(int(2e1)):
      params = model.init(random.PRNGKey(j), x_train, (x_train, y_train))
      rng = random.PRNGKey(j)
      rng, init_rng = random.split(rng)
      opt_state = optimizer.init(params, init_rng, (x_train, y_train))
      for i in range(int(2e3)):
        if i == 0:
          #print("j =", j)
          pass
        rng, step_rng = jax.random.split(rng)
        params, opt_state, stats = optimizer.step(
            params, opt_state, step_rng, batch=(x_train, y_train), global_step_int=i)
        if jnp.isnan(stats['loss']):
          break
        if (stats['loss'] < best_loss) and (stats['loss'] > 1e-12):
          best_params = jax.tree_map(lambda x: jnp.array(x), params)
          best_loss = stats['loss']
          if i % 25 == 0:
            sle_train = functional_form__sle(best_params, model, (x_train, y_train), (x_train, y_train))
            sle_test = functional_form__sle(best_params, model, (x_test, y_test), (x_train, y_train))
            #print(f"{jnp.sqrt(jnp.mean(sle_train)):.5e}", f"{jnp.sqrt(jnp.mean(sle_test)):.5e}", f"{best_loss:.5e}", "   ", f"{(time.time() - start_time):.5e}", "", j, i)

    bc = best_params
    print()
    print(title)
    print(bc)
    print("time:", time.time() - start_time)

    sle_all = functional_form__sle(bc, model, (x_all, y_all), (x_train, y_train))
    sle_train = functional_form__sle(bc, model, (x_train, y_train), (x_train, y_train))
    sle_test = functional_form__sle(bc, model, (x_test, y_test), (x_train, y_train))

    standard_le_all = functional_form__standard_le(bc, model, (x_all, y_all), (x_train, y_train))
    standard_le_train = functional_form__standard_le(bc, model, (x_train, y_train), (x_train, y_train))

    print("rmsle all:   ", jnp.sqrt(jnp.mean(sle_all)))
    print("rmsle train: ", jnp.sqrt(jnp.mean(sle_train)))
    print("rmsle test:  ", jnp.sqrt(jnp.mean(sle_test)))

    print("standard_le_all:   ", standard_le_all)
    print("standard_le_train: ", standard_le_train)

    data_dict["name"].append(plot_name)
    data_dict["rmsle_train"].append(jnp.sqrt(jnp.mean(sle_train)))
    data_dict["rmsle_extrap"].append(jnp.sqrt(jnp.mean(sle_test)))
    data_dict["rmsle_all"].append(jnp.sqrt(jnp.mean(sle_all)))
    data_dict["standard_le_train"].append(standard_le_train)
    data_dict["standard_le_all"].append(standard_le_all)

    points = 1024
    x_tile = jnp.expand_dims(jnp.logspace(-1, 15, points), -1)

    pred = model.apply(bc, x_tile, (x_train, y_train))

    plt.plot(x_train, y_train, 'o', color='black', markersize=9.1)
    plt.plot(x_test, y_test, 'o', color=[0.0, 0.835, 0.0], markersize=9.37, markerfacecolor=[0.0, 1.0, 0.0])
    plt.plot(x_tile, pred, color=[1.0, 0.1275, 0.1275], linewidth=2.15, alpha=1.0)

    plt.xlim(x.min()*.865,x.max()*1.05)
    plt.ylim(y.min()*.9,y.max()*1.05)

    plt.title(title, fontsize=16)
    plt.xlabel("Training Dataset Size", fontsize=12)
    plt.ylabel("Test Cross-Entropy", fontsize=12)
    plt.tick_params(axis='both', labelsize=12)

    """
    plt.xscale('log')
    plt.yscale('log')
    #"""

    plt.savefig('plots/'+plot_name+'.png', bbox_inches='tight')

    plt.show()

df = pd.DataFrame(data_dict)
df.to_csv("error.csv")

!zip -r plots.zip plots/